# Solving MNIST with ANN

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

/home/igor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data preparation
- importing data
- scaling data
- train-test splitting

In [82]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
mnist=fetch_mldata('MNIST original')
stdsc=StandardScaler()
#X,y=pd.DataFrame(mnist['data']), pd.DataFrame(mnist['target'])
X_all,y_all=mnist['data'], mnist['target'].astype(int)
X_all=stdsc.fit_transform(X_all)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

/home/igor/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [84]:
dataset= tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [13]:
mnist['target'],y,X_all

(array([0., 0., 0., ..., 9., 9., 9.]),
 array([0, 0, 0, ..., 9, 9, 9]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

### Defining the neural network for classification
- premade classificator DNNClassifier is used

In [80]:
tf.reset_default_graph()
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10
X=tf.placeholder(dtype=tf.float32,shape=(None, n_inputs), name="X")
y=tf.placeholder(shape=(None), dtype=tf.int32, name="y")

with tf.name_scope("DNN"):
    hidden1=tf.layers.dense(inputs=X, units=n_hidden1, activation=tf.nn.relu, name='hidden1')
    hidden2=tf.layers.dense(inputs=hidden1, units=n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits=tf.layers.dense(inputs=hidden2, units=n_outputs, activation=None, name='logits')
    
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy, name="loss")
    
learning_rate=0.01
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(targets=y,predictions=logits,k=1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
    
init=tf.global_variables_initializer()
saver=tf.train.Saver()

### Training the neural network:

In [4]:
n_epochs=40
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0]//batch_size):
            X_batch, y_batch=

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3df_oiga', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc9cdca3860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [81]:
X_train.shape

(56000, 784)

### Evaluating the Neural network on a test set

input function for test set (inputing in batsches in several epochs not needed now):

In [7]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": X_test}, y=y_test, shuffle=False)

evaluating results on the test set:

In [9]:
eval_results = dnnc.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-29-20:11:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3df_oiga/model.ckpt-44800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-29-20:11:06
INFO:tensorflow:Saving dict for global step 44800: accuracy = 0.97642857, average_loss = 0.16364935, global_step = 44800, loss = 20.828098


### Making predictions using neural network:

In [11]:
preds=dnnc.predict(input_fn=test_input_fn) # output is a generator
pred_results=list(preds) #saving all outputs of the generator to a list
pred_results[0] # predictions, logits probabilities for the first element of the test set

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3df_oiga/model.ckpt-44800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([-1.4758174, -2.3956823, -3.402472 , -8.597757 ,  5.104736 ,
        14.760276 , -6.009533 , 25.932945 ,  1.0617169, 10.407954 ],
       dtype=float32),
 'probabilities': array([1.24887709e-12, 4.97767734e-13, 1.81879671e-13, 1.00809404e-15,
        9.00362895e-10, 1.40528728e-05, 1.34137976e-14, 9.99985695e-01,
        1.57963313e-11, 1.80957201e-07], dtype=float32),
 'class_ids': array([7]),
 'classes': array([b'7'], dtype=object)}

predicted classes for all instances in the test set:

In [12]:
y_test_preds=np.array([p['class_ids'][0] for p in pred_results])

computing score using the predictions -matches to the score from evaluation part.

In [13]:
np.mean(y_test==y_test_preds)

0.9764285714285714